### リターンランキング
**【参考】**
[データ分析で振り返るマーケット2020 〜クラスター分析からポートフォリオ最適化まで〜](https://qiita.com/kenji_shinoda/items/ccf4f1366cef04231de9)

In [33]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [44]:
symbol_dict = {
    '^SPX':'S&P500',
    '^DJI':'ダウ平均',
    '^NKX':'日経平均',
    '^TPX':'TOPIX',
    'EURJPY':'ユーロ円',
    'USDJPY':'ドル円',
    # 'agg.us':'AGG',
    # '2510.jp':'2510',
    # 'GC.F':'GOLD',
    # 'btcusd':'BTC',
    # 'iyr.us':'IYR',
    # '%5Endq':'NASDAQ',
    # 'cl.f':'WTI',
    # '2042.jp':'Mothers',
    # 'SI.F':'Silver'
    }

In [45]:
symbols, names = np.array(list(symbol_dict.items())).T

df = pd.read_csv(f'https://stooq.com/q/d/l/?s={symbols[0]}&i=d',index_col=0)
df.index = pd.to_datetime(df.index).tz_localize('Asia/Tokyo')
df['variance'] = df['Close'] - df['Open']
df = df[['Close', 'variance']].copy()
df.columns = [names[0], names[0] + '_variance']

for i in range(1, len(symbols)):
    temp_df = pd.read_csv(f'https://stooq.com/q/d/l/?s={symbols[i]}&i=d',index_col=0)
    temp_df.index = pd.to_datetime(temp_df.index).tz_localize('Asia/Tokyo')
    temp_df['variance'] = temp_df['Close'] - temp_df['Open']
    temp_df = temp_df[['Close', 'variance']].copy()
    temp_df.columns = [names[i], names[i] + '_variance']

    # Dateをキーにして順次データを突合
    df = df.merge(temp_df, on = 'Date', how = 'inner')

In [50]:
result = []
for year in ['2018', '2019', '2020']:
    for name in names:
        year_df = df[name][year]

        return_start = year_df[-1]/year_df[0]
        return_min = year_df[-1]/np.min(year_df)
        variation_cof = np.std(year_df)/np.mean(year_df)

        result.append([year, name,return_start, return_min, variation_cof])

result_df = pd.DataFrame(result)
result_df.columns = ['year', 'name', 'return_start', 'return_min', 'variation_cof']

# 年初来リターンランキング
display(result_df.loc[result_df['year'] == '2020']
        .sort_values('return_start', ascending = False)
        [['name', 'return_start']].reset_index(drop = True).head(10))

# 最安値リターンランキング
display(result_df.loc[result_df['year'] == '2020']
        .sort_values('return_min', ascending = False)
        [['name', 'return_min']].reset_index(drop = True).head(10))

,name,return_start
0,日経平均,1.182691
1,S&P500,1.149636
2,TOPIX,1.063146
3,ダウ平均,1.059442
4,ユーロ円,1.046070
5,ドル円,0.952361


,name,return_min
0,S&P500,1.668025
1,日経平均,1.657974
2,ダウ平均,1.635632
3,TOPIX,1.459696
4,ユーロ円,1.102341
5,ドル円,1.007575
